In [11]:
import pandas as pd

test = pd.read_csv("../required_data_ec_number_paper/data/test.csv")
accessions = test["accession"]
terms = test.iloc[:, 8:].columns

/tmp/ipykernel_2665994/3818097993.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("../required_data_ec_number_paper/data/test.csv")


In [4]:
terms = list(terms)

In [4]:
test = test.iloc[:, :8]

In [5]:
# Create a function to format the EC columns
from tqdm import tqdm


def format_ec(df):
    accessions = []
    ecs_results = [] 
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        for i in range(1, 5):  # Loop through EC1 to EC4
            ecs = str(row[f'EC{i}']).split(";")
            for ec in ecs:
                accessions.append(f"{row['accession']}")
                ecs_results.append(f"EC:{ec}")
    return pd.DataFrame({
        "accessions": accessions,
        "EC": ecs_results

    })

# Apply the function to your dataframe
formatted_rows = format_ec(test)
formatted_rows.to_csv("gt.csv", index=False)

  0%|          | 0/75566 [00:00<?, ?it/s]

100%|██████████| 75566/75566 [00:08<00:00, 8724.95it/s]


In [9]:
def generate_obo(terms):
    obo_entries = []
    for i, term in enumerate(terms):
        # Create a unique id for each term (use IDPO with index)
        term_id = f"EC:{term}"  # IDPO: start from 1000
        name = f"Term {term}"  # Placeholder name for each term

        # Definition for each term (placeholder definition)
        definition = f'{term}'

        # Determine if it has a parent (is_a relationship)
        levels = term.split('.')
        if len(levels) == 1:
            parent = None  # Top-level term, no parent
        else:
            parent = ".".join(levels[:-1])  # Parent is the immediate higher level

        # OBO term format
        obo_entry = "[Term]\n"
        obo_entry += f"id: {term_id}\n"
        obo_entry += f"name: {name}\n"
        obo_entry += "namespace: ec_numbers\n"
        obo_entry += f"def: {definition}\n"

        # If a parent exists, add the 'is_a' field with the parent's ID
        if parent in terms:
            parent_index = terms.index(parent) # Get parent index
            obo_entry += f"is_a: EC:{parent}\n"

        obo_entries.append(obo_entry)

    return "\n".join(obo_entries)


# Generate the OBO file content
obo_content = generate_obo(terms)

# Write the content to a .obo file
with open("ontology_output.obo", "w") as obo_file:
    obo_file.write(obo_content)

print("OBO file generated successfully.")

OBO file generated successfully.


In [1]:
from plants_sm.io.pickle import read_pickle

test_esm1b = read_pickle("../pr_auc_validation/esm1b_predictions/test_esm1b_predictions.pkl")

In [9]:
import numpy as np

indexes = [np.where(row >= 0.02)[0] for row in test_esm1b]
terms = np.array(terms)

In [12]:
from tqdm import tqdm 
results = pd.DataFrame()

accessions_ = []
ec_list = []
prob = []
for i, index_ in tqdm(enumerate(indexes), total=75566):
    ecs = terms[index_]
    for j, ec in enumerate(ecs):
        accessions_.append(accessions[i])
        ec_list.append(f"EC:{ec}")
        prob.append(test_esm1b[i, index_[j]])

results = pd.DataFrame({
    "accession": accessions_,
    "EC": ec_list,
    "Prob": prob
})
results.to_csv("esm1b_results.tsv", index=False, sep="\t", header=False)

  0%|          | 0/75566 [00:00<?, ?it/s]

100%|██████████| 75566/75566 [00:01<00:00, 56925.29it/s]


In [2]:
import pandas as pd

pd.read_csv("gt.csv").to_csv("gt.tsv", sep="\t", header=False, index=False)